In [ ]:
!git clone https://github.com/peacefulmind111111/privacy-pipeline.git
%cd privacy-pipeline
%cd privacy_pipeline

In [ ]:
!pip install -r requirements.txt


In [ ]:
from dataclasses import asdict
from privacy_pipeline.config import ExperimentConfig
from privacy_pipeline.training import train, train_with_outlier_clipping


In [ ]:
baseline_cfg = ExperimentConfig(
    num_epochs=20,
    batch_size=512,
    lr=0.05,
    outer_momentum=0.9,
    inner_momentum=0.10,
    noise_mult=1.0,
    delta=1e-5,
    c_start=4.0,
    c_end=2.0,
    self_aug_factor=3,
    schedule_milestones=[12, 18],
    schedule_gamma=0.1,
    max_momentum_size=10000,
)
print(asdict(baseline_cfg))
train(baseline_cfg, 'outputs/baseline_run')


In [ ]:
outlier_cfg = ExperimentConfig(
    seed=0,
    batch_size=1000,
    lr=0.1,
    outer_momentum=0.9,
    inner_momentum=0.08,
    noise_mult=1.5,
    delta=1e-5,
    num_epochs=50,
    self_aug_factor=3,
    default_clip=1.0,
    outlier_clip=0.5,
    high_err_threshold=0.95,
    drop_after_frac=0.6,
    schedule_milestones=[30, 45],
    schedule_gamma=0.1,
    max_momentum_size=10000,
)
print(asdict(outlier_cfg))
train_with_outlier_clipping(outlier_cfg, 'outputs/outlier_run')
